## Problem Description

The aim is to add and correct bit flip and sign flip errors to a simple quantum circuit (Bell State circuit) which creates $|\psi\rangle$ = $\frac{|00\rangle + |11\rangle}{\sqrt{2}}$. Which implies when we measure the circuit we should get either $|00\rangle$ or $|11\rangle$ 

The **Bit Flip** error flips our qubit from 1 to 0 or vice versa. It is similar as applying a X gate.
<br> The **Sign Flip** error affects the phase of the qubit. In essence, it is equivalent to applying a Z gate.

We will have a total of 10 qubits which include 2 main qubits and 8 ancillary qubits used for corrections and 2 classical bits for measurement.
- To correct bit flip error of 1 qubit we require 2 ancillary qubits. 4 for our circuit.
- To correct sign flip error of 1 qubit we require 2 ancillary qubits. 4 for our circuit. 

In [ ]:
'''
    Required imports
'''
from qiskit import *
from qiskit.visualization import plot_histogram
import random
import numpy as np
import kaleidoscope.qiskit
from kaleidoscope import qsphere

The library <a href="https://nonhermitian.org/kaleido/index.html">kaleidoscope</a> will be used to plot the states along with their phase.<br>It can be installed using the command `pip install kaleidoscope`

### Initial Bell state

In [ ]:
q_bell = QuantumRegister(2, 'qb')
c_bell = ClassicalRegister(2, 'cb')
bell = QuantumCircuit(q_bell, c_bell)

bell.h(q_bell[0])
bell.cx(q_bell[0], q_bell[1])
bell.measure(q_bell, c_bell)

bell.draw(output='mpl')

#### Measurement of initial circuit

In [ ]:
qasm_back = Aer.get_backend('qasm_simulator')
sv_back = Aer.get_backend('statevector_simulator')

In [ ]:
shots = [100, 200, 500, 1000]
bell_res_counts = []
for shot in shots:
    bell_res = execute(bell, backend=qasm_back, shots = shot).result()
    bell_res_counts.append(bell_res.get_counts())
bell_res_counts

In [ ]:
legends = []
for shot in shots:
    legends.append(str(shot) + ' shots')

plot_histogram(bell_res_counts, legend = legends)

As we can see, the above circuit for Bell State produces the state, <center>$|\psi\rangle$ = $\frac{|00\rangle + |11\rangle}{\sqrt{2}}$ </center>with equal probability of measuring states $|00\rangle$ and $|11\rangle$

To also get the phase of the state we use the `statevector_simulator` and the library `kaleidoscope` to plot the state with  phase since the inbuilt `plot_state_qsphere` doesn't show the phase of the states properly.

The `kaleidoscope` library can be installed via the command `pip install kaleidoscope`

In [ ]:
bell_res_sv = {}
for shot in shots:
    bell_res_sphere = execute(bell, backend=sv_back).result()
    bell_res_sv[shot] = bell_res_sphere.get_statevector()
bell_res_sv

In [ ]:
qsphere(bell_res_sv[500])

### Adding errors

Next we will add errors to our qubits.
* For bit flip error, we will use `X` gate.
* For sign flip error, we will use `Z` gate.
* We will use `I` gate to simulate no error.

The way the error function works is, it takes in the quantum circuit (`qc`), the qubit (`q`) on which the error gate will be applied and a probability (`p`) as the input and based on the probability, it applies a specific gate. The function returns the name of the gate applied which will be used for the legends of the plots we will draw.
<br><br> The gates corresponding to the probability (p) are as follows:
- p ≤ 0.3 &emsp;&emsp;&emsp;Apply the Z gate
- 0.3 < p ≤ 0.6 &ensp;Apply the I gate
- p > 0.6 &emsp;&emsp;&emsp;Apply the X gate

Since the probability is randomly generated, it will cover all the combinations of error gates for the two qubits

In [ ]:
'''
    Function for introducing errors
    qc - our quantum circuit
    q - qubit to which error gate will be applied
    p - probability for applying a certain gate
    show - boolean flag to see the probability and gates applied
'''

def error(qc, q, p, show=True):
    gate = ''
    if(show):
        print("Probability value: ", p)
    if p <= 0.3: #apply the Z gate
        qc.z(q)
        if(show):
            print("Applied Z gate")
        gate = 'Z'
    elif p > 0.3 and p <= 0.6: #apply the I gate
        qc.i(q)
        if(show):
            print("Applied I gate")
        gate = 'I'
    else: #apply the X gate
        qc.x(q)
        if(show):
            print("Applied X gate")
        gate = 'X'
    return gate

Now, to check whether the `error` function works correctly, lets make the above Bell State circuit with error and take measurements.

In [ ]:
# initialisation
q_chk = QuantumRegister(2, 'q_chk')
c_chk = ClassicalRegister(2, 'c_chk')
qc_chk = QuantumCircuit(q_chk, c_chk)

qc_chk.h(q_chk[0])
qc_chk.barrier()

# adding error
# choosing probability for qubit 0
p = random.random()
error(qc_chk, q_chk[0], p)

# choosing probability for qubit 1
p = random.random()
error(qc_chk, q_chk[1], p)
qc_chk.barrier()

# remaining given circuit
qc_chk.cx(q_chk[0], q_chk[1])
qc_chk.barrier()

qc_chk.measure(q_chk, c_chk)

qc_chk.draw(output='mpl', justify='right')

We will execute the above circuit for, say 5 times, to check different outcomes of adding error

In [ ]:
times = 5
check_res_counts = []
check_res_sv = []
gates = {}

for i in range(times):
    applied_gate = ''
    # initialisation
    q_chk = QuantumRegister(2, 'q_chk')
    c_chk = ClassicalRegister(2, 'c_chk')
    qc_chk = QuantumCircuit(q_chk, c_chk)

    qc_chk.h(q_chk[0])
    qc_chk.barrier()

    # adding error
    # choosing probability for qubit 0
    p = random.random()
    applied_gate += error(qc_chk, q_chk[0], p)

    # choosing probability for qubit 1
    p = random.random()
    applied_gate += error(qc_chk, q_chk[1], p)
    gates[i] = applied_gate
    qc_chk.barrier()

    # remaining given circuit
    qc_chk.cx(q_chk[0], q_chk[1])
    qc_chk.barrier()

    qc_chk.measure(q_chk, c_chk)

    qc_chk.draw(output='mpl', justify='right')
    
    check_res = execute(qc_chk, backend=qasm_back).result()
    check_res_counts.append(check_res.get_counts())
    
    check_sv = execute(qc_chk, backend=sv_back).result().get_statevector()
    check_res_sv.append(check_sv)
check_res_counts

In [ ]:
legend = []
for i in range(times):
    legend.append(gates[i])

plot_histogram(check_res_counts, legend = legend, figsize = (15, 5))

We can see from the above plot that our bit flip error is working correctly! Now to see sign flip error, we will plot the qspheres of the above measurements.

In [ ]:
qsphere(check_res_sv[0])

In [ ]:
qsphere(check_res_sv[1])

In [ ]:
qsphere(check_res_sv[2])

In [ ]:
qsphere(check_res_sv[3])

In [ ]:
qsphere(check_res_sv[4])

From the above plots we can see that the sign flip error works as intended!

Now, let's correct the errors!!

In [ ]:
# initialisation
q = QuantumRegister(2, 'q')
anci_bit = QuantumRegister(4, 'anci_bit')
anci_sign = QuantumRegister(4, 'anci_sign')
c = ClassicalRegister(2, 'c')
qc = QuantumCircuit(q, anci_bit, anci_sign, c)

qc.h(q[0])
qc.barrier()

# transferring initial state of the qubits to ancillary qubits for sign flip correction
qc.cx(q[0], anci_sign[0])
qc.cx(q[0], anci_sign[1])

qc.cx(q[1], anci_sign[2])
qc.cx(q[1], anci_sign[3])
qc.h([q[0], anci_sign[0], anci_sign[1]])
qc.h([q[1], anci_sign[2], anci_sign[3]])
qc.barrier()

# transferring initial state of the qubits to ancillary qubits for bit flip correction
qc.cx(q[0], anci_bit[0])
qc.cx(q[0], anci_bit[1])

qc.cx(q[1], anci_bit[2])
qc.cx(q[1], anci_bit[3])
qc.barrier()

# adding error
# choosing probability for qubit 0
p = random.random()
error(qc, q[0], p)

# choosing probability for qubit 1
p = random.random()
error(qc, q[1], p)
qc.barrier()

# correcting bit flip
# for qubit 0
qc.cx(q[0], anci_bit[0])
qc.cx(q[0], anci_bit[1])
qc.ccx(anci_bit[0], anci_bit[1], q[0])

# for qubit 1
qc.cx(q[1], anci_bit[2])
qc.cx(q[1], anci_bit[3])
qc.ccx(anci_bit[2], anci_bit[3], q[1])
qc.barrier()

# correcting phase flip
# for qubit 0
qc.h([q[0], anci_sign[0], anci_sign[1]])
qc.h([q[1], anci_sign[2], anci_sign[3]])
qc.cx(q[0], anci_sign[0])
qc.cx(q[0], anci_sign[1])
qc.ccx(anci_sign[0], anci_sign[1], q[0])
qc.barrier()

# for qubit 1
qc.cx(q[1], anci_sign[2])
qc.cx(q[1], anci_sign[3])
qc.ccx(anci_sign[2], anci_sign[3], q[1])
qc.barrier()

# remaining given circuit
qc.cx(q[0], q[1])
qc.barrier()

qc.measure(q, c)

qc.draw(output='mpl', justify='right')

In [ ]:
shots = [100, 200, 500, 1000]
counts = []
sv_counts = {}

for shot in shots:
    result = execute(qc, backend=qasm_back, shots = shot).result()
    counts.append(result.get_counts())
    res_sv = execute(qc, backend=sv_back, shots = shot).result()
    sv_counts[shot] = res_sv.get_statevector()

In [ ]:
legends = []
for shot in shots:
    legends.append(str(shot) + ' shots')

plot_histogram(counts, legend = legends)

From the below plots, we can see that after correcting errors, all our final states have a phase of 0&deg;

In [ ]:
qsphere(sv_counts[100])

In [ ]:
qsphere(sv_counts[200])

In [ ]:
qsphere(sv_counts[500])

In [ ]:
qsphere(sv_counts[1000])

We can see from above, we get the desired output of the equal superposition state, <center>$|\psi\rangle$ = $\frac{|00\rangle + |11\rangle}{\sqrt{2}}$</center> after adding and correcting bit flip and sign flip errors.

Above we took measurements with different number of shots and the error gates were chosen by using random probability. Now let's check for all the different combinations of error and see whether our circuit can correct them.

In [ ]:
prob_vals = [0.2, 0.4, 0.8] # fixed probabilities for Z, I, X gates respectively

comb_counts = []
comb_sv_counts = {}
i = 0
for p0 in prob_vals:
    for p1 in prob_vals:
        # initialisation
        comb_q = QuantumRegister(2, 'q')
        comb_anci_bit = QuantumRegister(4, 'anci_bit')
        comb_anci_sign = QuantumRegister(4, 'anci_sign')
        comb_c = ClassicalRegister(2, 'c')
        comb_qc = QuantumCircuit(comb_q, comb_anci_bit, comb_anci_sign, comb_c)

        comb_qc.h(comb_q[0])
        comb_qc.barrier()

        # transferring initial state of the qubits to ancillary qubits for sign flip correction
        comb_qc.cx(comb_q[0], comb_anci_sign[0])
        comb_qc.cx(comb_q[0], comb_anci_sign[1])

        comb_qc.cx(comb_q[1], comb_anci_sign[2])
        comb_qc.cx(comb_q[1], comb_anci_sign[3])
        comb_qc.h([comb_q[0], comb_anci_sign[0], comb_anci_sign[1]])
        comb_qc.h([comb_q[1], comb_anci_sign[2], comb_anci_sign[3]])
        comb_qc.barrier()

        # transferring initial state of the qubits to ancillary qubits for bit flip correction
        comb_qc.cx(comb_q[0], comb_anci_bit[0])
        comb_qc.cx(comb_q[0], comb_anci_bit[1])

        comb_qc.cx(comb_q[1], comb_anci_bit[2])
        comb_qc.cx(comb_q[1], comb_anci_bit[3])
        comb_qc.barrier()

        # adding error
        # choosing probability for qubit 0
        error(comb_qc, comb_q[0], p0)

        # choosing probability for qubit 1
        error(comb_qc, comb_q[1], p1)
        comb_qc.barrier()

        # correcting bit flip
        # for qubit 0
        comb_qc.cx(comb_q[0], comb_anci_bit[0])
        comb_qc.cx(comb_q[0], comb_anci_bit[1])
        comb_qc.ccx(comb_anci_bit[0], comb_anci_bit[1], comb_q[0])

        # for qubit 1
        comb_qc.cx(comb_q[1], comb_anci_bit[2])
        comb_qc.cx(comb_q[1], comb_anci_bit[3])
        comb_qc.ccx(comb_anci_bit[2], comb_anci_bit[3], comb_q[1])
        comb_qc.barrier()

        # correcting phase flip
        # for qubit 0
        comb_qc.h([comb_q[0], comb_anci_sign[0], comb_anci_sign[1]])
        comb_qc.h([comb_q[1], comb_anci_sign[2], comb_anci_sign[3]])
        comb_qc.cx(comb_q[0], comb_anci_sign[0])
        comb_qc.cx(comb_q[0], comb_anci_sign[1])
        comb_qc.ccx(comb_anci_sign[0], comb_anci_sign[1], comb_q[0])
        comb_qc.barrier()

        # for qubit 1
        comb_qc.cx(comb_q[1], comb_anci_sign[2])
        comb_qc.cx(comb_q[1], comb_anci_sign[3])
        comb_qc.ccx(comb_anci_sign[2], comb_anci_sign[3], comb_q[1])
        comb_qc.barrier()

        # remaining given circuit
        comb_qc.cx(comb_q[0], comb_q[1])
        comb_qc.barrier()

        comb_qc.measure(comb_q, comb_c)        
        
        comb_counts.append(execute(comb_qc, backend=qasm_back).result().get_counts())
        comb_sv_counts[i] = execute(comb_qc, backend=sv_back).result().get_statevector()
        
        i += 1

comb_counts

In [ ]:
legend = []
for i in "ZIX":
    for j in "ZIX":
        legend.append(i+j)

plot_histogram(comb_counts, legend=legend, figsize=(16, 6))

Below we can see the phase of the states after measurement and check whether they align with our expected phase.

In [ ]:
qsphere(comb_sv_counts[0])

In [ ]:
qsphere(comb_sv_counts[1])

In [ ]:
qsphere(comb_sv_counts[2])

In [ ]:
qsphere(comb_sv_counts[3])

In [ ]:
qsphere(comb_sv_counts[4])

In [ ]:
qsphere(comb_sv_counts[5])

In [ ]:
qsphere(comb_sv_counts[6])

In [ ]:
qsphere(comb_sv_counts[7])

In [ ]:
qsphere(comb_sv_counts[8])

We can see that our circuit corrects all the different possible combinations of errors!

In essence the circuit looks something like this-
1. Transfer for bit flip correction
2. Transfer for sign flip correction
3. Apply error
4. Correct bit flip
5. Correct sign flip
7. Measurement

In [ ]:
def bit_transfer(q, anci_bit):
    fin_b = QuantumCircuit(q, anci_bit, name='transfer for\nbit correction')
    
    fin_b.cx(q[0], anci_bit[0])
    fin_b.cx(q[0], anci_bit[1])

    fin_b.cx(q[1], anci_bit[2])
    fin_b.cx(q[1], anci_bit[3])
    
    return fin_b

In [ ]:
def sign_transfer(q, anci_sign):
    fin_s = QuantumCircuit(q, anci_sign, name='transfer for\nsign correction')
    
    fin_s.cx(q[0], anci_sign[0])
    fin_s.cx(q[0], anci_sign[1])

    fin_s.cx(q[1], anci_sign[2])
    fin_s.cx(q[1], anci_sign[3])
    fin_s.h([q[0], anci_sign[0], anci_sign[1]])
    fin_s.h([q[1], anci_sign[2], anci_sign[3]])
    
    return fin_s

In [ ]:
def add_error(q):
    fin_e = QuantumCircuit(q, name='add\nerror')
    
    # choosing probability for qubit 0
    p = random.random()
    error(fin_e, q[0], p, False)

    # choosing probability for qubit 1
    p = random.random()
    error(fin_e, q[1], p, False)
    
    return fin_e

In [ ]:
def fix_bit(q, anci_bit):
    fin_fix_b = QuantumCircuit(q, anci_bit, name='fixing bit\nerror')
    
    # for qubit 0
    fin_fix_b.cx(q[0], anci_bit[0])
    fin_fix_b.cx(q[0], anci_bit[1])
    fin_fix_b.ccx(anci_bit[0], anci_bit[1], q[0])

    # for qubit 1
    fin_fix_b.cx(q[1], anci_bit[2])
    fin_fix_b.cx(q[1], anci_bit[3])
    fin_fix_b.ccx(anci_bit[2], anci_bit[3], q[1])
    
    return fin_fix_b

In [ ]:
def fix_sign(q, anci_sign):
    fin_fix_s = QuantumCircuit(q, anci_sign, name='fixing sign\nerror')
    
    # for qubit 0
    fin_fix_s.h([q[0], anci_sign[0], anci_sign[1]])
    fin_fix_s.cx(q[0], anci_sign[0])
    fin_fix_s.cx(q[0], anci_sign[1])
    fin_fix_s.ccx(anci_sign[0], anci_sign[1], q[0])

    # for qubit 1
    fin_fix_s.h([q[1], anci_sign[2], anci_sign[3]])
    fin_fix_s.cx(q[1], anci_sign[2])
    fin_fix_s.cx(q[1], anci_sign[3])
    fin_fix_s.ccx(anci_sign[2], anci_sign[3], q[1])
    
    return fin_fix_s

In [ ]:
# initialisation
fin_q = QuantumRegister(2, 'q')
fin_anci_bit = QuantumRegister(4, 'anci_bit')
fin_anci_sign = QuantumRegister(4, 'anci_sign')
fin_c = ClassicalRegister(2, 'c')
fin_qc = QuantumCircuit(q, anci_bit, anci_sign, c)

fin_qc.h(q[0])

# transferring initial state of the qubits to ancillary qubits for bit flip correction
fin_qc.append(bit_transfer(fin_q, fin_anci_bit), [0,1, 2,3,4,5])

# transferring initial state of the qubits to ancillary qubits for sign flip correction
fin_qc.append(sign_transfer(fin_q, fin_anci_sign), [0,1, 6,7,8,9])

# adding error
fin_qc.append(add_error(fin_q), [0,1])

# correcting bit flip
fin_qc.append(fix_bit(fin_q, fin_anci_bit), [0,1, 2,3,4,5])

# correcting sign flip
fin_qc.append(fix_sign(fin_q, fin_anci_sign), [0,1, 6,7,8,9])

# remaining given circuit
fin_qc.cx(fin_q[0], fin_q[1])

fin_qc.measure(fin_q, fin_c)

fin_qc.draw(output='mpl')